<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Tejas/Deep_Learning_HW3_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Homework 3

References:

Garbage collector: https://stackoverflow.com/questions/1316767/how-can-i-explicitly-free-memory-in-python

Checkpoint code and concept from : https://keras.io/api/callbacks/model_checkpoint/

In [1]:
#Path: TensorFlow/Tejas/
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


In [3]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
6.203125653999905
GPU (s):
0.06429144799994901
GPU speedup over CPU: 96x


##Dependencies

In [48]:
!pip install tensorflow-addons==0.12.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 703 kB 5.1 MB/s 
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [49]:
import tensorflow_addons as tfa

In [45]:
import numpy as np
import random
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import  Dense, Dropout, Activation, Flatten
from tensorflow.keras import optimizers
from librosa.core import stft,istft
import soundfile as sf
from sklearn.model_selection import train_test_split

import tensorflow as tf
%matplotlib inline

##Load Data Set

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cifar10_data = tf.keras.datasets.cifar10
(x_train, Y_train), (x_test, Y_test) = cifar10_data.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [6]:
x_train.shape

(50000, 32, 32, 3)

###Labelled and Unlabelled

In [7]:
X_train_label,X_train_unlabelled = x_train[49500:50000],x_train[0:49500]


In [8]:
y_train_label,y_train_unlabelled = Y_train[49500:50000],Y_train[0:49500]

##PreText Task

###Class 0 
Ref: https://www.tensorflow.org/api_docs/python/tf/zeros

In [9]:
X_train0 = tf.convert_to_tensor(X_train_unlabelled)

In [10]:
y_train0 = tf.zeros(shape = y_train_unlabelled.shape,dtype=tf.int32)

In [11]:
y_train0.shape # Just labels hence its (49500,1)

TensorShape([49500, 1])

In [12]:
print("shape of X : {} and Y is {}".format(X_train0.shape,y_train0.shape))

shape of X : (49500, 32, 32, 3) and Y is (49500, 1)


###Class 1 
Ref: 

https://www.tensorflow.org/api_docs/python/tf/image/flip_up_down

https://www.tensorflow.org/api_docs/python/tf/ones

In [13]:
X_train1 = tf.image.flip_up_down(X_train_unlabelled)

In [15]:
y_train1 = tf.ones(shape=y_train_unlabelled[0:49500].shape,dtype=tf.int32)

In [16]:
print("shape of X : {} and Y is {}".format(X_train1.shape,y_train1.shape))

shape of X : (49500, 32, 32, 3) and Y is (49500, 1)


In [17]:
y_train1

<tf.Tensor: shape=(49500, 1), dtype=int32, numpy=
array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)>

###Class 2

In [18]:
X_train2 = tf.image.rot90(X_train_unlabelled)

In [19]:
y_train2 = 2*tf.ones(shape=y_train_unlabelled[0:49500].shape,dtype=tf.int32)

In [20]:
print("shape of X : {} and Y is {}".format(X_train2.shape,y_train2.shape))


shape of X : (49500, 32, 32, 3) and Y is (49500, 1)


###Concat

In [21]:
X_train_merged = tf.concat([X_train0, X_train1,X_train2], 0)

In [22]:
y_train_merged = tf.concat([y_train0, y_train1,y_train2], 0)

###Pre Text Model



In [23]:
pretextmodel = keras.Sequential([
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last',input_shape=(32,32,3)),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last' ),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 20, activation = 'relu', kernel_initializer='he_normal'),
    keras.layers.Dense(units = 3, activation = 'softmax', kernel_initializer = 'he_normal')
])

In [24]:
pretextmodel.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [25]:
pretextmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 10)        760       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 10)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 10)        2510      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 10)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 250)               0         
                                                                 
 dense (Dense)               (None, 20)                5

Checkpoint code and concept from : https://keras.io/api/callbacks/model_checkpoint/

In [26]:
history = pretextmodel.fit(X_train_merged, y_train_merged, batch_size=128, epochs=100)

Epoch 1/100
1161/1161 [==============================] - 13s 4ms/step - loss: 1.4403 - accuracy: 0.3470
Epoch 2/100
1161/1161 [==============================] - 5s 4ms/step - loss: 1.0499 - accuracy: 0.4168
Epoch 3/100
1161/1161 [==============================] - 5s 4ms/step - loss: 0.9475 - accuracy: 0.5340
Epoch 4/100
1161/1161 [==============================] - 5s 4ms/step - loss: 0.9096 - accuracy: 0.5685
Epoch 5/100
1161/1161 [==============================] - 5s 4ms/step - loss: 0.8792 - accuracy: 0.5919
Epoch 6/100
1161/1161 [==============================] - 5s 4ms/step - loss: 0.8617 - accuracy: 0.6051
Epoch 7/100
1161/1161 [==============================] - 5s 5ms/step - loss: 0.8449 - accuracy: 0.6164
Epoch 8/100
1161/1161 [==============================] - 5s 4ms/step - loss: 0.8310 - accuracy: 0.6253
Epoch 9/100
1161/1161 [==============================] - 5s 4ms/step - loss: 0.8182 - accuracy: 0.6345
Epoch 10/100
1161/1161 [==============================] - 5s 4ms/step - 

In [27]:
pretextmodel.save('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Models/pretext_model.h5')

##Baseline Model

In [30]:
baselinemodel = keras.Sequential([
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last',input_shape=(32,32,3)),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last' ),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 20, activation = 'relu', kernel_initializer='he_normal'),
    keras.layers.Dense(units = 10, activation = 'softmax', kernel_initializer = 'he_normal')
])

In [33]:
baselinemodel.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


In [34]:
baselinemodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 28, 28, 10)        760       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 10)       0         
 2D)                                                             
                                                                 
 conv2d_27 (Conv2D)          (None, 10, 10, 10)        2510      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 10)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 250)               0         
                                                                 
 dense_4 (Dense)             (None, 20)               

In [35]:
baselinehistory = baselinemodel.fit(X_train_label, 
          y_train_label,  
          epochs=10000,
          validation_data = (x_test, Y_test),
          verbose=1,
          batch_size=256)

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 161ms/step - loss: 6.3681e-07 - accuracy: 1.0000 - val_loss: 26.1160 - val_accuracy: 0.1949
Epoch 7502/10000
2/2 [==============================] - 0s 166ms/step - loss: 6.3586e-07 - accuracy: 1.0000 - val_loss: 26.1179 - val_accuracy: 0.1949
Epoch 7503/10000
2/2 [==============================] - 0s 173ms/step - loss: 6.3443e-07 - accuracy: 1.0000 - val_loss: 26.1191 - val_accuracy: 0.1950
Epoch 7504/10000
2/2 [==============================] - 0s 165ms/step - loss: 6.3419e-07 - accuracy: 1.0000 - val_loss: 26.1196 - val_accuracy: 0.1949
Epoch 7505/10000
2/2 [==============================] - 0s 340ms/step - loss: 6.3300e-07 - accuracy: 1.0000 - val_loss: 26.1197 - val_accuracy: 0.1949
Epoch 7506/10000
2/2 [==============================] - 0s 334ms/step - loss: 6.3324e-07 - accuracy: 1.0000 - val_loss: 26.1208 - val_accuracy: 0.1949
Epoch 7507/10000
2/2 [==============================] - 0s 1

In [36]:
baselinemodel.save('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Models/baseline_model.h5')

##Transfer Learning

Ref and code: https://www.tensorflow.org/js/guide/models_and_layers

Layer wise optimizer customization: https://www.tensorflow.org/addons/api_docs/python/tfa/optimizers/MultiOptimizer

In [30]:
TL_model = tf.keras.models.load_model('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Models/pretext_model.h5')

In [31]:
TL_model.pop()

In [34]:
TL_model.add(Dense(10, activation='softmax', kernel_initializer='he_normal'))

In [35]:
optimizers = [
    tf.keras.optimizers.Adam(learning_rate=1e-6),
    tf.keras.optimizers.Adam(learning_rate=1e-3)
]

In [36]:
optimizers_and_layers = [(optimizers[0], TL_model.layers[:-1]), (optimizers[1], TL_model.layers[-1])]

In [51]:
optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

AttributeError: ignored